In [1]:
# Line magic functions that will allow for imports to be reloaded and not cached
%load_ext autoreload
%autoreload 2

# Imports
import os
import pandas as pd
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.azure_openai import AzureOpenAI
from datetime import date
from json import dump
from tqdm import tqdm

# Local
from scripts.extractmd import Extractor
from scripts.vectorindex import QnAEngine
from scripts.utilities import get_prompt_dict, get_questions, get_answers, get_procurement_content, get_config_data, get_ini_files, get_supplementary_info, get_questions_without_q0
from scripts.gen_results import gen_results
from scripts.gen_reports import generate_precision_report

**My Config:**
[my_config.py](scripts/my_config.py)

In [2]:
with open("scripts/my_config.py") as f:
    code = f.read()
    print(code) 
    exec(code)  

# -----------------------------------------------------------------------------
# Configuration file: my_config.py
# -----------------------------------------------------------------------------
# This file is used to define **user-specific configuration parameters**
# separately from the Jupyter notebook. It allows each user to set their own
# values without modifying the shared notebook or interfering with others.
#
# Usage pattern in the notebook:
#     overwrite = globals().get('my_overwrite', False)
#
# This means:
# - If `my_overwrite` is defined in this config file, its value will be used.
# - If it is not defined, the default value (`False` in this case) will be used.
#
# You can define any custom variable here that the notebook expects via `globals().get(...)`.
#
#  In Jupyter, this script may be executed multiple times.
# Commenting out a variable won't reset it if it was previously defined.
# To revert to the default (e.g., via globals().get(...)), use:
#     del my_variable

**Global config**

In [3]:
embedding_conf = {
    "embeddingmodel": globals().get('my_embeddingmodel', "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"), 
        #"sentence-transformers/paraphrase-multilingual-mpnet-base-v2",  # "BAAI/bge-m3" "nomic-ai/nomic-embed-text-v2-moe" # "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
    "chunk_size": 1536,
    "chunk_overlap": 0,
    "top_similar": 5,
    "n4rerank": 0, #How many nodes to retrieve for reranking. If 0, reranker is not used
    "use_similar_chunks": True, #To use similar chunks or the whole document as the context
    "prevnext": True #to include in the context also the previouse and the next chunk of the current similar chunk
}
embedding=HuggingFaceEmbedding(model_name=embedding_conf["embeddingmodel"],trust_remote_code=True)

#For nomic-embed-text-v2-moe
#embedding=HuggingFaceEmbedding(model_name=embedding_conf["embeddingmodel"],trust_remote_code=True,query_instruction="search_query: ",text_instruction="search_document: ")

In [4]:
# LLM Setup 
llmmodelAzure = { "model": "gpt-4o",
                "version":os.environ.get('AZURE_OPENAI_VERSION',''),
                "azure_deployment":"gpt-4o",
                "azure_endpoint":os.environ.get('AZURE_ENDPOINT',''),
                "api_key":os.environ.get('AZURE_OPENAI_KEY','')}

llm=AzureOpenAI(azure_deployment=llmmodelAzure["azure_deployment"],
                azure_endpoint=llmmodelAzure["azure_endpoint"],temperature=0.0,
                api_version=llmmodelAzure["version"], api_key=llmmodelAzure["api_key"],
                timeout=120,max_retries=3,top_p=0.0001)

In [5]:
extractor = Extractor() # Markdown doc extractor

In [6]:
#Ollama model
# llmmodelOllama = { "model": "gemma3:27b",
#                 "url":os.environ.get('OLLAMA_ENDPOINT',''),
#                 "context_window":"128000"}

#from llama_index.llms.ollama import Ollama
#llm = Ollama(base_url=llmmodelOllama["url"],
#             model=llmmodelOllama["model"], 
#             context_window=int(llmmodelOllama["context_window"]),
#            request_timeout=300.0,
#            temperature=0.0,
#            additional_kwargs={"seed":1337})

**PROCUREMENT FILE SETTINGS**

In [7]:
# Script dir for getting relative paths for notebook file
script_dir = globals()['_dh'][0] 

# Document paths
question_file_path = script_dir / "questions" / "questions.yaml" # original.yaml
prompt_file = script_dir / "questions" / "prompts.tsv"
report_dir = script_dir / "reports"
config_dir = script_dir / globals().get('my_config_dir', "dev_config") # "dev_config" # "dev_config" # "config"
procurement_file_dir = script_dir / "cfla_files" # "cfla_files"
answer_file_dir = script_dir / "answers"

# TODO perhaps prompt user to define unique report name; some types - all; one etc?
report_identifier = globals().get('my_report_identifier', "dev-test")
# TODO maybe add report as a subdirectory as there are 2 files per report; might be even more with histograms etc.
report_name = f"{report_identifier}_{date.today():%d.%m}"

report_dir_path = report_dir / report_name
report_path_htm = report_dir_path / "report.htm"
report_path_csv = report_dir_path / "report.csv"
report_path_config = report_dir_path / "config.json"

In [8]:
# Loading static information
overwrite = globals().get('my_overwrite', False)  
            # If true this will delete the existing report and generate a new one;
            # Else - new data will be appended only if it isn't in the CSV file.

question_dictionary = get_questions(question_file_path)
prompt_dictionary = get_prompt_dict(prompt_file)
supplementary_info = get_supplementary_info()

ini_files = get_ini_files(config_dir, overwrite, report_path_csv)
print(f"Processing {len(ini_files)} procurement files: {sorted(ini_files)}")

Questions loaded
Found 10 config files in C:\Repos\vpp-cfla\dev_config
Processing 10 procurement files: ['KND-2020_07', 'KND-2020_20', 'KNP202134', 'LNP_202050ERAF', 'LU_CFI_201935ERAF', 'RPNC202122', 'SNP-2021_07_AK', 'SNP202131', 'SNP_20213ERAF', 'VNIP_2020_036_ERAF']


**MAIN Q/A GENERATION SCRIPT**

In [9]:
# TODO add parallel prompting

if overwrite: # overwrtitting report; Delete and create new
        if report_path_htm.exists():
                report_path_htm.unlink()
        if report_path_csv.exists():
                report_path_csv.unlink()
        if report_path_config.exists():
                report_path_config.unlink()
                
if not os.path.exists(report_dir_path):
        os.makedirs(report_dir_path)

# Make config dictionary and save as json
config_dict = embedding_conf
config_dict["model"] = llmmodelAzure["model"]
config_dict["temperature"] = llm.temperature
config_dict["timeout"] = llm.timeout
config_dict["max_retries"] = llm.max_retries
#config_dict["top_p"] = 0.0001

with open(report_path_config, "w") as file:
        dump(config_dict, file) 

# To process all questions, leave this empty. Otherwise, specify the exact numbers.
# To process a sub-question, include the parent question number as well, e.g., ["9", "9.2"].
questions_to_process = globals().get('my_questions_to_process', [])

for file in tqdm(ini_files, desc="Config files", unit="file"): # TODO fix TQDM; expected that after first file finishes it shows the aproximation of all the other files left to process
        configfile = config_dir / f"{file}.ini"
        
        tqdm.write(f"Processing config file: {configfile}")
        procurement_id, procurement_file, agreement_file, answer_file = get_config_data(configfile, procurement_file_dir, answer_file_dir)
        # Open CSV file, maybe as pandas dataframe
        answer_dictionary = get_answers(answer_file)

        # Getting markdown text from procurement doc
        procurement_content = get_procurement_content(extractor, procurement_file, agreement_file)
    
        # Creating FAISS vector index for the procurement document
        qnaengine = QnAEngine(embedding,llm)
        await qnaengine.createIndex(
                procurement_content,
                "Procurement",
                chunk_size=embedding_conf["chunk_size"],
                chunk_overlap=embedding_conf["chunk_overlap"]
                )

        ### Generating results
        results_table = gen_results(qnaengine, configfile, embedding_conf, question_dictionary, answer_dictionary, prompt_dictionary, supplementary_info, questions_to_process)
        
        # add "Iepirkuma ID" as procurement_id to results table
        # TODO move this inside gen results function once it has been refactored
        for row in results_table:
                row.insert(0, file)
        
        ### Save output
        data = pd.DataFrame(results_table, columns=["Iepirkuma ID", "Nr", "Atbilde", "Sagaidāmā atbilde", "Pamatojums", "Prompt"])
        precision = (data['Atbilde'] == data['Sagaidāmā atbilde']).sum()/len(data)
        print(f"PRECIZITĀTE: {precision*100}%")

        with report_path_htm.open('a', encoding='utf-8') as ofile:  
                # TODO Create a dropdown menu that lets the user select file by "Iepirkuma ID" - in each page only information that has that "Iepirkuma ID" is displayed
                print(f"{procurement_id} PRECIZITĀTE: {precision*100}%" ,file=ofile)
                print(data.to_html(index=False).replace('\\n','<br>'),file=ofile)

        data.to_csv(report_path_csv, 
                    mode='a', 
                    index=False, 
                    header=not report_path_csv.exists(), # only adding one header
                    encoding='utf-8')
        
        
                

Config files:   0%|                                                                           | 0/10 [00:00<?, ?file/s]

Processing config file: C:\Repos\vpp-cfla\dev_config\KND-2020_07.ini



Generating embeddings:   0%|                                                                           [ time left: ? ]
Generating embeddings:  20%|██████████████                                                         [ time left: 01:20 ]
Generating embeddings:  38%|██████████████████████████▋                                            [ time left: 00:59 ]
Generating embeddings:  56%|███████████████████████████████████████▍                               [ time left: 00:54 ]
Generating embeddings:  75%|████████████████████████████████████████████████████▏                  [ time left: 00:28 ]
Generating embeddings: 100%|██████████████████████████████████████████████████████████████████████ [ time left: 00:00 ]


55 segments created and vectorized.
Index is ready.
2 4 6 7 9 10 15 16 17 18 19 22 23 24 26 27 28 29 31 32 33 34 35 35.1 35.2 35.3 35.5 35.6 35.7 35.8 36 37 37.2 37.4 37.5 37.6 37.7 37.9 37.10 37.11 37.13 38 38.1 38.2 38.3 38.4 38.5 38.6 38.7 38.8 38.9 39 39.1 39.2 39.3 39.4 39.5 39.6 39.7 39.8 39.9 39.10 39.11 39.12 39.13 39.14 39.15 39.16 39.17 39.18 39.19 39.20 39.21 39.22 39.23 39.24 39.25 39.26 39.27 39.28 39.29 39.30 39.31 39.32 39.33 39.34 39.35 39.36 39.37 39.38 39.39 40 40.1 40.2 40.3 40.4 40.5 40.6 40.7 40.8 40.9 40.10 40.11 40.12 41 43 44 46 47 48 50 51 51.1 51.2 51.4 52 53 54 56 57 58 60 61 62 63 65 

Config files:  10%|██████▍                                                         | 1/10 [10:32<1:34:49, 632.19s/file]

...

PRECIZITĀTE: 66.07142857142857%
Processing config file: C:\Repos\vpp-cfla\dev_config\VNIP_2020_036_ERAF.ini



Generating embeddings:   0%|                                                                           [ time left: ? ]
Generating embeddings:  22%|███████████████▋                                                       [ time left: 01:00 ]
Generating embeddings:  43%|██████████████████████████████                                         [ time left: 00:47 ]
Generating embeddings:  63%|████████████████████████████████████████████▎                          [ time left: 00:38 ]
Generating embeddings: 100%|██████████████████████████████████████████████████████████████████████ [ time left: 00:00 ]


49 segments created and vectorized.
Index is ready.
2 4 6 7 9 10 15 16 17 18 19 22 23 24 26 27 28 29 31 32 33 34 35 35.1 35.2 35.3 35.5 35.6 35.7 35.8 36 37 37.2 37.4 37.5 37.6 37.7 37.9 37.10 37.11 37.13 38 38.1 38.2 38.3 38.4 38.5 38.6 38.7 38.8 38.9 39 39.1 39.2 39.3 39.4 39.5 39.6 39.7 39.8 39.9 39.10 39.11 39.12 39.13 39.14 39.15 39.16 39.17 39.18 39.19 39.20 39.21 39.22 39.23 39.24 39.25 39.26 39.27 39.28 39.29 39.30 39.31 39.32 39.33 39.34 39.35 39.36 39.37 39.38 39.39 40 40.1 40.2 40.3 40.4 40.5 40.6 40.7 40.8 40.9 40.10 40.11 40.12 41 43 44 46 47 48 50 51 51.1 51.2 51.4 52 53 54 56 57 58 60 61 62 63 65 

Config files: 100%|███████████████████████████████████████████████████████████████| 10/10 [1:59:47<00:00, 718.74s/file]

PRECIZITĀTE: 54.166666666666664%


**Question precision data report**

In [10]:
# Load report we want to analayze
analyzed_report_dir = report_dir_path
csv_filename = "report.csv"
input_csv = analyzed_report_dir / csv_filename

In [11]:
# Generate report
precison_report_html = generate_precision_report(input_csv)

# Save the report next to the CSV
output_html = analyzed_report_dir / f"precision_report.html"
with open(output_html, "w", encoding="utf-8") as f:
    f.write(precison_report_html)

print(f"HTML report saved to: {output_html}")

HTML report saved to: C:\Repos\vpp-cfla\reports\dev-bge-m3_17.07\precision_report.html


**Questions that can't be answered count**

In [12]:
# Count number of times expected_answer == "n/a; if question does not have 0th question
questions_wout_0q = get_questions_without_q0(question_dictionary)
df = pd.read_csv(input_csv)
df["Sagaidāmā atbilde"] = df["Sagaidāmā atbilde"].astype(str).str.strip().str.lower()
filtered_df = df[
    (df["Sagaidāmā atbilde"] == "nan") &
    (df["Nr"].astype(str).isin(questions_wout_0q))
]
print(f"\nCount of rows where 'Sagaidāmā atbilde' == 'n/a' and it doesn't have q0: {len(filtered_df)}")


Count of rows where 'Sagaidāmā atbilde' == 'n/a' and it doesn't have q0: 165


**Confusion Matrix**

In [13]:
# Load and normalize data as "n/a" has turned into NaN
df = pd.read_csv(input_csv)
df["Sagaidāmā atbilde"] = df["Sagaidāmā atbilde"].astype(str).str.strip().str.lower()
df["Atbilde"] = df["Atbilde"].astype(str).str.strip().str.lower()

# Replace missing values
df["Sagaidāmā atbilde"] = df["Sagaidāmā atbilde"].replace(["nan", "none", ""], "n/a")
df["Atbilde"] = df["Atbilde"].replace(["nan", "none", ""], "n/a")

# Group context responses
df["Atbilde"] = df["Atbilde"].replace({
    "x": "kontekstā nav informācijas",
    "kontekstā nav informācijas": "kontekstā nav informācijas"
})

# Define value order
expected = ["jā", "nē", "n/a"]
actual = ["jā", "nē", "n/a", "kontekstā nav informācijas"]

conf_matrix = pd.crosstab(
    df["Sagaidāmā atbilde"],
    df["Atbilde"],
    rownames=["Expected ↓"],
    colnames=["Actual →"],
    dropna=False
).reindex(index=expected, columns=actual, fill_value=0)
conf_matrix.index.name = "Expected ↓ / Actual →"

print("\nConfusion Matrix:\n")
print(conf_matrix.to_markdown(tablefmt="grid"))



Confusion Matrix:

+-------------------------+------+------+-------+------------------------------+
| Expected ↓ / Actual →   |   jā |   nē |   n/a |   kontekstā nav informācijas |
+=========================+======+======+=======+==============================+
| jā                      |  686 |  119 |     8 |                           69 |
+-------------------------+------+------+-------+------------------------------+
| nē                      |   74 |  169 |     1 |                           78 |
+-------------------------+------+------+-------+------------------------------+
| n/a                     |  111 |   48 |   178 |                          119 |
+-------------------------+------+------+-------+------------------------------+


In [14]:
# Get correct counts for "jā" and "nē"
correct_yes = conf_matrix.loc["jā", "jā"]
total_yes = conf_matrix.loc["jā"].sum()
print(f"total 'jā' count: {total_yes}")

correct_no = conf_matrix.loc["nē", "nē"]
total_no = conf_matrix.loc["nē"].sum()
print(f"total 'nē' count: {total_no}")

correct_na = conf_matrix.loc["n/a", "n/a"]
total_na = conf_matrix.loc["n/a"].sum()
print(f"total 'n/a' count: {total_na}")

# Calculate percentages
yes_accuracy = (correct_yes / total_yes) * 100 if total_yes > 0 else 0
no_accuracy = (correct_no / total_no) * 100 if total_no > 0 else 0
na_accuracy = (correct_na / total_na) * 100 if total_na > 0 else 0

print(f"\nAccuracy for 'jā': {yes_accuracy:.2f}%")
print(f"Accuracy for 'nē': {no_accuracy:.2f}%")
print(f"Accuracy for 'n/a': {na_accuracy:.2f}%")


total 'jā' count: 882
total 'nē' count: 322
total 'n/a' count: 456

Accuracy for 'jā': 77.78%
Accuracy for 'nē': 52.48%
Accuracy for 'n/a': 39.04%


In [15]:
total_q = total_yes + total_no + total_na
print(f"Total question count: {total_q}")
correct_q = correct_yes + correct_no + correct_na

precision = round((correct_q / total_q) * 100, 2)
print(f"Total precision: {precision}%")

precison_wout_unanswerable_qs = round(correct_q / (total_q - len(filtered_df))*100,2)
print(f"Total precision without unanswerable (n/a) questions: {precison_wout_unanswerable_qs}%")

context_missing_total = conf_matrix["kontekstā nav informācijas"].sum()
confident_answers =  total_q - len(filtered_df) - context_missing_total
precision_wout_unanswerable_qs_nocontext = round((correct_q / confident_answers)*100,2)
print(f"Total precision without unanswerable (n/a) questions and when LLM is not unsure: {precision_wout_unanswerable_qs_nocontext}%")
print(f"Total question count without unanswerable (n/a) questions and when LLM is not unsure: {confident_answers}")

Total question count: 1660
Total precision: 62.23%
Total precision without unanswerable (n/a) questions: 69.1%
Total precision without unanswerable (n/a) questions and when LLM is not unsure: 84.05%
Total question count without unanswerable (n/a) questions and when LLM is not unsure: 1229
